In [3]:
from sklearn.model_selection import KFold
from lightgbm import LGBMRegressor
from sklearn.compose import make_column_transformer, make_column_selector, TransformedTargetRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import KFold


from ssri_interactions.config import ExperimentInfo, Config
from ssri_interactions.io import load_derived_generic
from ssri_interactions.decoding.loaders import FSDecodeDataLoader, FSFastDecodeDataLoader
from ssri_interactions.decoding.preprocessors import DecodePreprocessor
from ssri_interactions.decoding.encoders import StateEncoder
from ssri_interactions.decoding.runners import EncodeRunner
from ssri_interactions.decoding.shuffle import shuffle_X

c:\ProgramData\Miniconda3\envs\ssri\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [4]:
def make_slow_ts_runner(loader):
    preprocessor = DecodePreprocessor(
    thresh_empty=2,
    )

    ct = make_column_transformer(
        (
            StandardScaler(),
            make_column_selector(dtype_exclude=object)
        ),  
        (
            OneHotEncoder(drop="if_binary"),
            make_column_selector(dtype_include=object)
        ), 
    )

    estimator = make_pipeline(ct, 
        LGBMRegressor(
            n_estimators=20, 
            n_jobs=-1, 
            force_row_wise=True,
            reg_lambda=0.8,
            )
    )

    estimator = TransformedTargetRegressor(estimator, transformer=StandardScaler())
    cv = KFold(shuffle=True)
    encoder = StateEncoder(estimator=estimator, cv=cv, verbose=True)
    runner = EncodeRunner(
        loader=loader,
        preprocessor=preprocessor,
        encoder=encoder,
    )
    return runner

In [5]:
neuron_types = load_derived_generic("neuron_types.csv").query("group in ('CIT', 'SAL')")
sessions = neuron_types.query("experiment_name == 'HAMILTON'").session_name.unique().tolist()
session = sessions[0]
loader_shock_only = FSDecodeDataLoader(session_name=session, t_stop=600)
loader_post_included = FSFastDecodeDataLoader(session_name=session, t_stop=1200)

In [6]:
runner_shock_only = make_slow_ts_runner(loader_shock_only)

pop = runner_shock_only.run_multiple_pop(sessions=sessions)
pop_shuffle = runner_shock_only.run_multiple_pop(sessions=sessions, shuffle=True)
state = runner_shock_only.run_multiple_state(sessions=sessions)
state_shuffle = runner_shock_only.run_multiple_state(sessions=sessions, shuffle=True)
limit = runner_shock_only.run_multiple_limit(sessions=sessions, min_features=1, max_features=15)
dropout = runner_shock_only.run_multiple_dropout(sessions=sessions)

dd = Config.derived_data_dir / "encoding"
dd.mkdir(exist_ok=True)
pop.to_csv(dd / "fs_slow - shock_only - pop.csv", index=False)
pop_shuffle.to_csv(dd / "fs_slow - shock_only - pop shuffle.csv", index=False)
state.to_csv(dd / "fs_slow - shock_only - state.csv", index=False)
state_shuffle.to_csv(dd / "fs_slow - shock_only - state shuffle.csv", index=False)
limit.to_csv(dd / "fs_slow - shock_only - limit.csv", index=False)
dropout.to_csv(dd / "fs_slow - shock_only - dropout.csv", index=False)
    

  0%|          | 0/19 [00:00<?, ?it/s]c:\ProgramData\Miniconda3\envs\ssri\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
c:\ProgramData\Miniconda3\envs\ssri\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
c:\ProgramData\Miniconda3\envs\ssri\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
  5%|▌         | 1/19 [00:01<00:29,  1.64s/it]c:\ProgramData\Miniconda3\envs\ssri\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
c:\ProgramData\Miniconda3\envs\ssri\lib\site-packages\sklearn\feature_selection\_univariate_selectio

In [8]:
runner_post_included = make_slow_ts_runner(loader_post_included)

pop = runner_post_included.run_multiple_pop(sessions=sessions)
pop_shuffle = runner_post_included.run_multiple_pop(sessions=sessions, shuffle=True)
state = runner_post_included.run_multiple_state(sessions=sessions)
state_shuffle = runner_post_included.run_multiple_state(sessions=sessions, shuffle=True)
limit = runner_post_included.run_multiple_limit(sessions=sessions, min_features=1, max_features=15)
dropout = runner_post_included.run_multiple_dropout(sessions=sessions)

dd = Config.derived_data_dir / "encoding"
dd.mkdir(exist_ok=True)
pop.to_csv(dd / "fs_slow - post_included - pop.csv", index=False)
pop_shuffle.to_csv(dd / "fs_slow - post_included - pop shuffle.csv", index=False)
state.to_csv(dd / "fs_slow - post_included - state.csv", index=False)
state_shuffle.to_csv(dd / "fs_slow - post_included - state shuffle.csv", index=False)
limit.to_csv(dd / "fs_slow - post_included - limit.csv", index=False)
dropout.to_csv(dd / "fs_slow - post_included - dropout.csv", index=False)

 24%|██▎       | 4/17 [00:09<00:30,  2.38s/it]c:\ProgramData\Miniconda3\envs\ssri\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
 53%|█████▎    | 9/17 [00:21<00:19,  2.40s/it]c:\ProgramData\Miniconda3\envs\ssri\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
  0%|          | 0/19 [00:00<?, ?it/s]c:\ProgramData\Miniconda3\envs\ssri\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
c:\ProgramData\Miniconda3\envs\ssri\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:290: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= np.linalg.norm(y)
c:\ProgramData\Miniconda3\envs\ssri\lib\site

In [9]:
loader = FSFastDecodeDataLoader(session_name=session)
preprocessor = DecodePreprocessor(
    thresh_empty=2,
)
spikes, states = loader()
spikes, states = preprocessor(spikes, states)

ct = make_column_transformer(
    (
        StandardScaler(),
        make_column_selector(dtype_exclude=object)
    ),  
    (
        OneHotEncoder(drop="if_binary"),
        make_column_selector(dtype_include=object)
    ), 
)

estimator = make_pipeline(ct, 
    LGBMRegressor(
        n_estimators=20, 
        n_jobs=-1, 
        force_row_wise=True,
        reg_lambda=0.8,
        )
)

estimator = TransformedTargetRegressor(estimator, transformer=StandardScaler())
cv = KFold(shuffle=True)
encoder = StateEncoder(estimator=estimator, cv=cv, verbose=True)
runner = EncodeRunner(
    loader=loader,
    preprocessor=preprocessor,
    encoder=encoder,
)

In [10]:
pop = runner.run_multiple_pop(sessions=sessions)
pop_shuffle = runner.run_multiple_pop(sessions=sessions, shuffle=True)
state = runner.run_multiple_state(sessions=sessions)
state_shuffle = runner.run_multiple_state(sessions=sessions, shuffle=True)
limit = runner.run_multiple_limit(sessions=sessions, min_features=1, max_features=15)
dropout = runner.run_multiple_dropout(sessions=sessions)

 18%|█▊        | 3/17 [00:07<00:34,  2.43s/it]c:\ProgramData\Miniconda3\envs\ssri\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
 47%|████▋     | 8/17 [00:20<00:22,  2.53s/it]c:\ProgramData\Miniconda3\envs\ssri\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
 82%|████████▏ | 14/17 [00:34<00:07,  2.39s/it]c:\ProgramData\Miniconda3\envs\ssri\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
  0%|          | 0/19 [00:00<?, ?it/s]c:\ProgramData\Miniconda3\envs\ssri\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
c:\Prog

In [11]:
dd = Config.derived_data_dir / "encoding"
dd.mkdir(exist_ok=True)
pop.to_csv(dd / "fs_fast - pop.csv", index=False)
state.to_csv(dd / "fs_fast - state.csv", index=False)
limit.to_csv(dd / "fs_fast - limit.csv", index=False)
dropout.to_csv(dd / "fs_fast - dropout.csv", index=False)
    